In [1]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list, parse_file_to_event_list, parse_raw_file_to_event_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

cn = kb.ConceptNet()  # choose your KB 
cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

events = parse_file_to_event_list("datasets/output_feb25.txt")  # This reads in the file. 
additional_facts = create_facts_from_file("gaze_facts.txt") # should call rules 
cn_monitor.explain_events(events, additional_facts)


Made a ConceptNet monitor, now trying to explain
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining 

Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking i

Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in up direction
Explaining looking
  robot is observing a looking event

Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking i

  Since user is looking in right direction, user is looking at the picture
  Left THEN Right
Explaining final
  robot is observing a final event
  user spoke "he"
  user speaks, so user may want help.
Explaining final
  robot is observing a final event
  user spoke "nd"
  user speaks, so user may want help.
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
  Right THEN Center
Explaining final
  robot is observing a final event
  user spoke "nd"
  user speaks, so user may want help.
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining final
  robot is observing a final event
  user spoke "he"
  user speaks, so user may want help.
Explaining lo

...

  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction,

Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking i

  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction,

Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking i

  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction,

In [4]:
pwd

'/Users/leilani/workspace/anomaly-explain'

In [3]:
from commonsense.logical_classes import Fact, Event

def test():
    file1 = open("/Users/leilani/workspace/anomaly-explain/datasets/allnewsemanticdatawithmultimodal/allnewsemanticdatawithmultimodal0025.txt", 'r')
    lines = file1.readlines()
    events = []
    lookingPointCounter = 1
    lookingDirectionCounter = 1
    speaksCounter = 1

    count = 0
    # Strips the newline character
    for line in lines:  # each one is a new event
        tokens = line.strip().replace("(", "").replace(")", "").split()
#         print(tokens)
        if tokens[2] == 'lookingAtPoint':
            eventName = 'lookingAtPEvent%d'%lookingPointCounter
            [Fact(eventName, 'isa', 'LookingAtPointEvent')]
            facts = make_looking_event_from_line(eventName, 'lookingAtPoint', 
                                               tokens[1], tokens[3], tokens[4], tokens[5], tokens[6], tokens[7])
            event = Event(facts, tokens[7])
            events.append(event)
            lookingPointCounter += 1
        elif tokens[2] == 'lookingInDirection':
            eventName = 'lookingAtPEvent%d'%lookingDirectionCounter
            facts = make_direction_event_from_line(eventName, 'lookingInDirection', 
                                               tokens[1], tokens[3], tokens[4], tokens[5])
            event = Event(facts, tokens[5])
            events.append(event)
            lookingDirectionCounter += 1
        elif tokens[2] == 'speaksUtterance': # utterance
            # isa speaksUttEvent23 speaksUtteranceEvent
            eventName = 'speaksUttEvent%d'%speaksCounter
            facts = make_utterance_event_from_line(eventName, 'speaksUtteranceEvent', 
                                            tokens[1], tokens[3], tokens[4], tokens[5])
            event = Event(facts, tokens[5])
            events.append(event)
            speaksCounter += 1
        else:
            # (labelEndsAt (needsLevelOfHelp user 1) 1698)
            facts = [Fact(tokens[2], tokens[1], tokens[3]),
                    Fact(eventName, 'timestamp', tokens[4])]
            event = Event(facts, tokens[4])
            events.append(event)
    return events

def make_looking_event_from_line(eventName, eventType, observer, looker, x, y, z, timestamp):
    return [Fact(eventName, 'isa', eventType),
           Fact(eventName, 'observedBy', observer),
           Fact(eventName, 'looker', looker),
           Fact(eventName, 'pointX', x), 
           Fact(eventName, 'pointY', y), 
           Fact(eventName, 'pointZ', z),
           Fact(eventName, 'timestamp', timestamp)]

def make_direction_event_from_line(eventName, eventType, observer, looker, direction, timestamp):
    return [Fact(eventName, 'isa', eventType),
           Fact(eventName, 'observedBy', observer),
           Fact(eventName, 'looker', looker),
           Fact(eventName, 'direction', direction), 
           Fact(eventName, 'timestamp', timestamp)]

def make_utterance_event_from_line(eventName, eventType, observer, speaker, utterance, timestamp):
    return [Fact(eventName, 'isa', eventType),
           Fact(eventName, 'observedBy', observer),
           Fact(eventName, 'speaker', speaker),
           Fact(eventName, 'utterance', utterance), 
           Fact(eventName, 'timestamp', timestamp)]
            
# Raw data
events = test()  # This reads in the file. 
events

cn_monitor.explain_events(events, additional_facts)

Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in center direction
  Since user is looking in center direction, user is looking at the robot
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking e

Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
  user looking in downright direction
  Down THEN DownRight
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a looking event
Explaining looking
  robot is observing a lo

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

cn = kb.ConceptNet()  # choose your KB 
cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

facts = parse_file_to_fact_list("datasets/output_feb25.txt", 30)  # This reads in the file. 
additional_facts = create_facts_from_file("gaze_facts.txt")

cn_monitor.explain_all_events(facts, additional_facts)

In [ ]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

cn = kb.ConceptNet()  # choose your KB 
cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

facts = parse_file_to_fact_list("datasets/output_feb25.txt")  # This reads in the file. 
additional_facts = create_facts_from_file("gaze_facts.txt")

cn_monitor.explain_all_events(facts, additional_facts)

In [ ]:
facts

f = to_data_frame(facts)
events = f.groupby("subject")
f.groupby("subject").count()
events.first()
events.nth(0)

facts